<a href="https://colab.research.google.com/github/eliaskosiski/Aula2---FIB/blob/master/Classificador_de_Spam_com_Naive_Bayes_e_SKLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
from sklearn.naive_bayes import BernoulliNB

In [25]:
import os

NEWLINE = '\n'
SKIP_FILES = set(['cmds'])


def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))

    for file_name in file_names:
        if file_name not in SKIP_FILES:
            file_path = os.path.join(root, file_name)

        if os.path.isfile(file_path):
            past_header, lines = False, []

            #f = open(file_path)

            f = open(file_path, mode="r", encoding="Latin-1")
            for line in f:
                if past_header:
                    lines.append(line)

                elif line == NEWLINE:
                    past_header = True
            f.close()


            yield file_path, NEWLINE.join(lines)#.decode('cp1252', 'ignore')
            

In [26]:
from pandas import DataFrame

def build_data_frame(path, classification):
  data_frame = DataFrame({'text': [], 'class': []})


  for file_name, text in read_files(path):
    data_frame = data_frame.append(
        DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
    

  return data_frame
  

In [ ]:
HAM = 0
SPAM = 1


SOURCES = [
      ('/content/drive/MyDrive/spamassassin/spam', SPAM),
      ('/content/drive/MyDrive/spamassassin/hard_ham', HAM),
      ('/content/drive/MyDrive/spamassassin/easy_ham', HAM)
    ]


data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
  data = data.append(build_data_frame(path, classification))

  
data = data.reindex(numpy.random.permutation(data.index))


In [ ]:
count_vectorizer = CountVectorizer()

counts = count_vectorizer.fit_transform(numpy.asarray(data['text']))


In [ ]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()

targets = numpy.asarray(data['class'])

classifier.fit(counts, targets)


In [ ]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]

example_counts = count_vectorizer.transform(examples)


predictions = classifier.predict(example_counts)
predictions # [1, 0]


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
  ('vectorizer',  CountVectorizer()),
  ('classifier',  MultinomialNB()) ])


pipeline.fit(numpy.asarray(data['text']), numpy.asarray(data['class']))
pipeline.predict(examples) # [1, 0]


In [28]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),

  ('classifier',         BernoulliNB(binarize=0.0)) ])


In [29]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),

  ('classifier',         BernoulliNB(binarize=0.0)) ])


In [30]:
data

,text,class
